In [1]:
%%html
<div style="text-align: center; background: #1a1a1a; padding: 20px; border-radius: 15px; color: white; font-family: sans-serif;">
    <h3>MediaPipe Holistic (Cuerpo + Manos + Rostro Detallados)</h3>
    <video id="v_pose_pro" width="640" height="480" style="display:none" playsinline></video>
    <canvas id="c_pose_pro" width="640" height="480" style="background: #000; border: 2px solid #9b59b6; border-radius: 10px;"></canvas>
    
    <div style="margin-top: 15px;">
        <button id="btn_on_pose" onclick="iniciarPosePro()" style="padding: 10px 20px; background: #9b59b6; color: white; border: none; border-radius: 5px; cursor: pointer; font-weight: bold;">🚀 ACTIVAR DETECCIÓN</button>
        <button id="btn_off_pose" onclick="detenerTodo()" style="padding: 10px 20px; background: #e74c3c; color: white; border: none; border-radius: 5px; cursor: pointer; margin-left: 10px; display: none;">🛑 APAGAR</button>
    </div>
    
    <div style="margin-top: 10px; display: flex; justify-content: center; gap: 5px; flex-wrap: wrap;">
        <button onclick="togglePart('body')" style="padding: 8px 15px; background: #3498db; color: white; border: none; border-radius: 5px; cursor: pointer;">🧍 Cuerpo</button>
        <button onclick="togglePart('hands')" style="padding: 8px 15px; background: #f39c12; color: white; border: none; border-radius: 5px; cursor: pointer;">✋ Manos (21 puntos)</button>
        <button onclick="togglePart('face')" style="padding: 8px 15px; background: #e74c3c; color: white; border: none; border-radius: 5px; cursor: pointer;">👤 Rostro (468 puntos)</button>
    </div>
    
    <p id="log_pose_pro" style="color: #f1c40f; font-size: 13px; margin-top: 10px;">Estado: Esperando cámara...</p>
    <p style="color: #95a5a6; font-size: 11px; margin-top: 5px;">💡 Tip: Acerca tu mano o rostro para ver detalle máximo</p>
</div>

<script src="https://cdn.jsdelivr.net/npm/@mediapipe/camera_utils/camera_utils.js" crossorigin="anonymous"></script>
<script src="https://cdn.jsdelivr.net/npm/@mediapipe/drawing_utils/drawing_utils.js" crossorigin="anonymous"></script>
<script src="https://cdn.jsdelivr.net/npm/@mediapipe/holistic/holistic.js" crossorigin="anonymous"></script>

<script>
// Variables de configuración
let visibleParts = {
    body: true,
    hands: true,
    face: true
};

let holisticCamera = null;

function togglePart(part) {
    visibleParts[part] = !visibleParts[part];
}

async function iniciarPosePro() {
    const log = document.getElementById('log_pose_pro');
    
    detenerTodo();
    
    ['c_canny_pro', 'c_out', 'c_sobel_pro', 'c_pose_pro'].forEach(canvasId => {
        const canvas = document.getElementById(canvasId);
        if (canvas) {
            const ctx = canvas.getContext('2d');
            ctx.clearRect(0, 0, canvas.width, canvas.height);
            ctx.fillStyle = '#000';
            ctx.fillRect(0, 0, canvas.width, canvas.height);
        }
    });
    
    try {
        log.innerText = "📷 Solicitando cámara...";
        
        const stream = await navigator.mediaDevices.getUserMedia({ 
            video: { width: 640, height: 480, facingMode: 'user' } 
        });
        
        window.cameraManager.currentStream = stream;
        window.cameraManager.currentFilter = 'pose';
        
        const v = document.getElementById('v_pose_pro');
        const canvas = document.getElementById('c_pose_pro');
        const ctx = canvas.getContext('2d');
        
        v.srcObject = stream;
        await v.play();
        
        document.getElementById('btn_on_pose').style.display = 'none';
        document.getElementById('btn_off_pose').style.display = 'inline-block';
        
        log.innerText = "⏳ Cargando MediaPipe Holistic (cuerpo completo + manos + rostro)...";
        
        // Usar HOLISTIC para obtener TODO con alta precisión
        const holistic = new Holistic({
            locateFile: (file) => {
                return `https://cdn.jsdelivr.net/npm/@mediapipe/holistic/${file}`;
            }
        });
        
        holistic.setOptions({
            modelComplexity: 2,
            smoothLandmarks: true,
            enableSegmentation: false,
            smoothSegmentation: false,
            refineFaceLandmarks: true, // ¡CLAVE! Rostro detallado
            minDetectionConfidence: 0.5,
            minTrackingConfidence: 0.5
        });
        
        let frameCount = 0;
        
        holistic.onResults((results) => {
            if (window.cameraManager.currentFilter !== 'pose') return;
            
            ctx.save();
            ctx.clearRect(0, 0, canvas.width, canvas.height);
            ctx.drawImage(results.image, 0, 0, canvas.width, canvas.height);
            
            let detections = [];
            
            // ========== CUERPO (33 puntos) ==========
            if (visibleParts.body && results.poseLandmarks) {
                // Esqueleto del cuerpo
                drawConnectors(ctx, results.poseLandmarks, POSE_CONNECTIONS, {
                    color: '#00FFFF',
                    lineWidth: 4
                });
                
                // Puntos del cuerpo
                drawLandmarks(ctx, results.poseLandmarks, {
                    color: '#00FFFF',
                    fillColor: '#00FFFF',
                    lineWidth: 2,
                    radius: 5
                });
                
                detections.push('Cuerpo (33 puntos)');
            }
            
            // ========== MANOS IZQUIERDA (21 puntos cada una) ==========
            if (visibleParts.hands) {
                // Mano izquierda
                if (results.leftHandLandmarks) {
                    drawConnectors(ctx, results.leftHandLandmarks, HAND_CONNECTIONS, {
                        color: '#FFD700',
                        lineWidth: 3
                    });
                    drawLandmarks(ctx, results.leftHandLandmarks, {
                        color: '#FFD700',
                        fillColor: '#FFFF00',
                        lineWidth: 2,
                        radius: 4
                    });
                    detections.push('Mano Izq (21 puntos)');
                }
                
                // Mano derecha
                if (results.rightHandLandmarks) {
                    drawConnectors(ctx, results.rightHandLandmarks, HAND_CONNECTIONS, {
                        color: '#FF8C00',
                        lineWidth: 3
                    });
                    drawLandmarks(ctx, results.rightHandLandmarks, {
                        color: '#FF8C00',
                        fillColor: '#FFA500',
                        lineWidth: 2,
                        radius: 4
                    });
                    detections.push('Mano Der (21 puntos)');
                }
            }
            
            // ========== ROSTRO (468 puntos - ¡SÍ, 468!) ==========
            if (visibleParts.face && results.faceLandmarks) {
                // Malla facial completa
                drawConnectors(ctx, results.faceLandmarks, FACEMESH_TESSELATION, {
                    color: '#FF1493',
                    lineWidth: 0.5
                });
                
                // Contornos importantes del rostro
                drawConnectors(ctx, results.faceLandmarks, FACEMESH_RIGHT_EYE, {
                    color: '#00FF00',
                    lineWidth: 2
                });
                drawConnectors(ctx, results.faceLandmarks, FACEMESH_LEFT_EYE, {
                    color: '#00FF00',
                    lineWidth: 2
                });
                drawConnectors(ctx, results.faceLandmarks, FACEMESH_LIPS, {
                    color: '#FF0000',
                    lineWidth: 2
                });
                drawConnectors(ctx, results.faceLandmarks, FACEMESH_FACE_OVAL, {
                    color: '#E0E0E0',
                    lineWidth: 2
                });
                
                // Puntos clave del rostro
                drawLandmarks(ctx, results.faceLandmarks, {
                    color: '#FF1493',
                    fillColor: '#FF69B4',
                    lineWidth: 1,
                    radius: 1
                });
                
                detections.push('Rostro (468 puntos)');
            }
            
            frameCount++;
            if (frameCount % 30 === 0) {
                const detectedText = detections.length > 0 ? detections.join(' + ') : 'Esperando...';
                log.innerText = `✅ ${detectedText} | Frames: ${frameCount}`;
            }
            
            ctx.restore();
        });
        
        // Iniciar cámara
        holisticCamera = new Camera(v, {
            onFrame: async () => {
                if (window.cameraManager.currentFilter === 'pose') {
                    await holistic.send({image: v});
                }
            },
            width: 640,
            height: 480
        });
        
        await holisticCamera.start();
        log.innerText = "✅ Detección completa activa - Muestra tus manos y rostro";
        
    } catch (e) {
        log.innerText = "❌ Error: " + e.message;
        console.error(e);
    }
}
</script>